In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
import seaborn as sns

In [ ]:
tf.keras.backend.clear_session()

In [ ]:
#Adding Seed so that random initialization is consistent
import os
from numpy.random import seed
seed(1)
from tensorflow import set_random_seed
set_random_seed(2)

In [ ]:
import dataset

#Prepare input data
train_path='training_data'
classes = ['cats']
print(classes)
num_classes = len(classes)

# 10% of the data will automatically be used for validation
validation_size = 0.1
img_size = 48
num_channels = 3
sample_size = 8192

data = dataset.read_train_sets(train_path, img_size, ['cats'], validation_size=validation_size, sample_size=sample_size)


print("Complete reading input data. Will Now print a snippet of it")
print("Number of files in Training-set:\t\t{}".format(len(data.train.labels)))
print("Number of files in Validation-set:\t{}".format(len(data.valid.labels)))

## Creating the AutoEncoder with three hidden layers

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers

total_pixels = img_size * img_size * 3
translator_factor = 2
translator_layer_size = int(total_pixels/translator_factor)
middle_factor = 2
middle_layer_size = int(translator_layer_size/middle_factor)

inputs = keras.Input(shape=(img_size,img_size,3), name='cat_image')
x = layers.Flatten(name = 'flattened_cat')(inputs) #49152 #3072
x = layers.BatchNormalization()(x)
x = layers.Dense(translator_layer_size, activation='relu', name='encoder')(x)
x = layers.Dense(middle_layer_size, activation='relu', name='middle_layer')(x)
x = layers.BatchNormalization()(x)
x = layers.Dense(translator_layer_size, activation='relu', name='decoder')(x)

outputs = layers.Dense(total_pixels, activation='sigmoid', name='reconstructed_cat')(x)
outputs = layers.Reshape((img_size,img_size,3))(outputs)

model = keras.Model(inputs=inputs, outputs=outputs)

In [ ]:
customAdam = keras.optimizers.Adam(lr=0.001)
model.compile(optimizer=customAdam,  # Optimizer
              # Loss function to minimize
              loss="mean_squared_error",
              # List of metrics to monitor
              metrics=["mean_squared_error"])

## Training the model

In [ ]:
x_train, _, _, _ = data.train.next_batch(7373)
x_valid, _, _, _ = data.valid.next_batch(819)

In [ ]:
print('# Fit model on training data')

history = model.fit(x_train, 
                    x_train, #we pass it the input itself as desired output
                    batch_size=256,
                    epochs=2,
                    # We pass it validation data to
                    # monitor loss and metrics
                    # at the end of each epoch
                    validation_data=(x_valid, x_valid))

# visual

In [ ]:
import matplotlib.pyplot as plt

valid_predictions = model.predict(x_valid[:40])
for index in range(20):
    instance = x_valid[index]
    decoded_img = valid_predictions[index]


    plt.figure(figsize=(20, 4))
    subplot = plt.subplot(2, 10, 1)
    plt.imshow(instance)

    subplot.get_xaxis().set_visible(False)
    subplot.get_yaxis().set_visible(False)

    subplot = plt.subplot(2, 10, 2)
    plt.imshow(decoded_img)

    subplot.get_xaxis().set_visible(False)
    subplot.get_yaxis().set_visible(False)